### **1. Import Libraries**

In [261]:
import pandas as pd
import re

In [262]:
# Langkah 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **2. Membaca Dataset Kotor**

In [263]:
# Baca file .xlsx
file_path = '/content/drive/MyDrive/Colab Notebooks/Messy_Employee_Dataset.xlsx'
data = pd.read_excel(file_path, engine='openpyxl')  # Gunakan openpyxl untuk file .xlsx

In [264]:
# Tampilkan data
print(data)

     Employee_ID First_Name Last_Name   Age   Department_Region    Status  \
0        EMP1000        Bob     Davis  25.0   DevOps-California    Active   
1        EMP1001        Bob     Brown   NaN       Finance-Texas    Active   
2        EMP1002      Alice     Jones   NaN        Admin-Nevada   Pending   
3        EMP1003        Eva     Davis  25.0        Admin-Nevada  Inactive   
4        EMP1004      Frank  Williams  25.0  Cloud Tech-Florida    Active   
...          ...        ...       ...   ...                 ...       ...   
1015     EMP2015      David    Miller  30.0       HR-California    Active   
1016     EMP2016      David   Johnson  30.0    Cloud Tech-Texas  Inactive   
1017     EMP2017    Charlie  Williams  40.0    Finance-New York    Active   
1018     EMP2018      Alice    Garcia  30.0          HR-Florida  Inactive   
1019     EMP2019      Heidi     Jones  30.0     DevOps-Illinois   Pending   

      Join_Date     Salary                         Email       Phone  \
0  

### **3. Menampilkan Jumlah Data, Kolom, Dan Tipe Data**

In [265]:
# Baca dataset
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Messy_Employee_Dataset.xlsx")

# Tampilkan informasi dataset
print("=== Informasi Dataset ===")

# Jumlah baris (data)
print("Jumlah Data (baris):", df.shape[0])

# Jumlah kolom
print("Jumlah Kolom:", df.shape[1])

# Tipe data tiap kolom
print("\nTipe Data Setiap Kolom:")
print(df.dtypes)

=== Informasi Dataset ===
Jumlah Data (baris): 1020
Jumlah Kolom: 12

Tipe Data Setiap Kolom:
Employee_ID                  object
First_Name                   object
Last_Name                    object
Age                         float64
Department_Region            object
Status                       object
Join_Date            datetime64[ns]
Salary                      float64
Email                        object
Phone                         int64
Performance_Score            object
Remote_Work                    bool
dtype: object


### **4. Data Cleansing - Standarisasi Kolom First_Name & Last_Name**

In [266]:
# Fungsi untuk membersihkan nama
def clean_name(name):
    if pd.isna(name):                  # Jika datanya kosong (NaN)
        return "Unknown"               # Ganti dengan "Unknown"
    name = str(name).strip()           # Hapus spasi di depan & belakang
    name = re.sub(r"\s+", " ", name)   # Ganti spasi berlebih jadi satu
    return name.title()                # Ubah huruf awal tiap kata jadi kapital

# Terapkan ke kolom nama
df["First_Name"] = df["First_Name"].apply(clean_name)
df["Last_Name"] = df["Last_Name"].apply(clean_name)
df[['First_Name', 'Last_Name']].head()

,First_Name,Last_Name
0,Bob,Davis
1,Bob,Brown
2,Alice,Jones
3,Eva,Davis
4,Frank,Williams


### **5. Data Cleansing - Standarisasi Kolom Age**

In [267]:
# Isi nilai kosong (NaN) pada kolom Age dengan nilai median (nilai tengah)
df["Age"] = df["Age"].fillna(df["Age"].median())
df[['Age']].head()

,Age
0,25.0
1,30.0
2,30.0
3,25.0
4,25.0


### **6. Data Cleansing - Standarisasi Kolom Department_Region**

In [268]:
# Memisahkan Departmen_Region menjadi 2 kolom

# Pastikan kolom Department_Region sudah bersih dulu
df["Department_Region"] = df["Department_Region"].fillna("Unknown-Unknown")
df["Department_Region"] = df["Department_Region"].str.strip()

# Pisahkan kolom berdasarkan tanda "-"
df[["Department", "Region"]] = df["Department_Region"].str.split("-", n=1, expand=True)

# Hapus kolom Department_Region karena sudah dipisah
df = df.drop(columns=["Department_Region"])

# Bersihkan hasil split (hapus spasi, kapitalisasi)
df["Department"] = df["Department"].str.strip().str.title()
df["Region"] = df["Region"].str.strip().str.title()
df[['Department', 'Region']].head()

,Department,Region
0,Devops,California
1,Finance,Texas
2,Admin,Nevada
3,Admin,Nevada
4,Cloud Tech,Florida


### **7. Data Cleansing - Standarisasi Kolom Status**

In [269]:
# Isi NaN dengan "Unknown"
df["Status"] = df["Status"].fillna("Unknown")

# Hapus spasi berlebih & kapitalisasi
df["Status"] = df["Status"].str.strip().str.title()
df[['Status']].head()

,Status
0,Active
1,Active
2,Pending
3,Inactive
4,Active


### **8. Data Cleansing - Standarisasi Kolom Performance_Score**

In [270]:
# Normalisasi huruf besar/kecil
df["Performance_Score"] = df["Performance_Score"].str.strip().str.title()
df[['Performance_Score']].head()

,Performance_Score
0,Average
1,Excellent
2,Good
3,Good
4,Poor


### **9. Data Cleansing - Standarisasi Kolom Email**

In [271]:
# Fungsi untuk validasi email
def validate_email(email):
    if pd.isna(email):                            # Jika kosong
        return "invalid@example.com"
    pattern = r"^[\w\.-]+@[\w\.-]+\.\w+$"         # Pola regex email
    return email if re.match(pattern, str(email)) else "invalid@example.com"

# Terapkan fungsi ke kolom Email
df["Email"] = df["Email"].apply(validate_email)
df[['Email']].head()

,Email
0,bob.davis@example.com
1,bob.brown@example.com
2,alice.jones@example.com
3,eva.davis@example.com
4,frank.williams@example.com


### **10. Data Cleansing - Standarisasi Kolom Phone**

In [272]:
# Fungsi untuk membersihkan nomor telepon
def clean_phone(phone):
    phone = re.sub(r"\D", "", str(phone))   # Hapus semua karakter selain angka
    return phone

# Terapkan fungsi ke kolom Phone
df["Phone"] = df["Phone"].apply(clean_phone)

# Hapus baris dengan Phone < 10 digit
df = df[df["Phone"].str.len() >= 10]
df[['Phone']].head()

,Phone
0,1651623197
1,1898471390
2,5596363211
3,3476490784
4,1586734256


### **11. Data Cleansing - Standarisasi Kolom Join_Date**

In [273]:
# Pastikan kolom Join_Date sudah dalam format datetime
df["Join_Date"] = pd.to_datetime(df["Join_Date"], errors="coerce")

# Ubah format tampilan jadi DD-MM-YYYY
df["Join_Date"] = df["Join_Date"].dt.strftime("%d / %m / %Y")
df[['Join_Date']].head()

,Join_Date
0,02 / 04 / 2021
1,10 / 07 / 2020
2,07 / 12 / 2023
3,27 / 11 / 2021
4,05 / 01 / 2022


### **12. Data Cleansing - Standarisasi Kolom Salary**

In [274]:
# Konversi ke angka (float), jika error → NaN
df["Salary"] = pd.to_numeric(df["Salary"], errors="coerce")

# Hapus baris yang Salary-nya kosong (NaN)
df = df.dropna(subset=["Salary"])
df[['Salary']].head()

,Salary
0,59767.65
1,65304.66
2,88145.90
3,69450.99
4,109324.61


### **13. Data Cleansing - Standarisasi Kolom Employee_ID**

In [275]:
# Reset & buat Employee_ID berurutan (AMAN)

# Backup Employee_ID lama supaya kodenya "tidak dihapus"
df["Employee_ID_old"] = df["Employee_ID"].astype(str)

# Reset index agar penomoran berjalan dari 0..n-1 secara kontinyu
# Tanpa reset index, baris yang sebelumnya di-drop/ difilter bisa punya index "kosong"
df = df.reset_index(drop=True)

# Buat Employee_ID baru berurutan mulai dari 1 (format EMP0001, EMP0002, ...)
df["Employee_ID"] = ["EMP" + str(i+1).zfill(4) for i in range(len(df))]

# Cek hasil
print("Duplikat Employee_ID setelah reset:", df["Employee_ID"].duplicated().sum())
display(df[["Employee_ID_old", "Employee_ID"]].head(5))

Duplikat Employee_ID setelah reset: 0


,Employee_ID_old,Employee_ID
0,EMP1000,EMP0001
1,EMP1001,EMP0002
2,EMP1002,EMP0003
3,EMP1003,EMP0004
4,EMP1004,EMP0005


In [276]:
# mengubah urutan kolom
kolom_urutan = [
    "Employee_ID",
    "First_Name",
    "Last_Name",
    "Age",
    "Phone",
    "Email",
    "Join_Date",
    "Department",
    "Region",
    "Salary",
    "Status",
    "Performance_Score",
    "Remote_Work"
]

# Susun ulang DataFrame sesuai urutan kolom
df = df[kolom_urutan]
print(df.head())

  Employee_ID First_Name Last_Name   Age       Phone  \
0     EMP0001        Bob     Davis  25.0  1651623197   
1     EMP0002        Bob     Brown  30.0  1898471390   
2     EMP0003      Alice     Jones  30.0  5596363211   
3     EMP0004        Eva     Davis  25.0  3476490784   
4     EMP0005      Frank  Williams  25.0  1586734256   

                        Email       Join_Date  Department      Region  \
0       bob.davis@example.com  02 / 04 / 2021      Devops  California   
1       bob.brown@example.com  10 / 07 / 2020     Finance       Texas   
2     alice.jones@example.com  07 / 12 / 2023       Admin      Nevada   
3       eva.davis@example.com  27 / 11 / 2021       Admin      Nevada   
4  frank.williams@example.com  05 / 01 / 2022  Cloud Tech     Florida   

      Salary    Status Performance_Score  Remote_Work  
0   59767.65    Active           Average         True  
1   65304.66    Active         Excellent         True  
2   88145.90   Pending              Good         True  


### **14. Data Cleansing - Deduplikasi Data**

In [277]:
duplicate_rows = df[df.duplicated(['Employee_ID', 'First_Name', 'Last_Name', 'Age', 'Phone', 'Email', 'Join_Date', 'Department', 'Region', 'Salary', 'Status', 'Performance_Score', 'Remote_Work'], keep='first')]
print("Jumlah data yang duplikat:", len(duplicate_rows))
duplicate_rows #menampilkan data yang duplikat

Jumlah data yang duplikat: 0


,Employee_ID,First_Name,Last_Name,Age,Phone,Email,Join_Date,Department,Region,Salary,Status,Performance_Score,Remote_Work


### **15. Data Cleansing - Data Setelah Cleansing Duplikasi**

In [278]:
df_cleaned = df.drop_duplicates(['Employee_ID', 'First_Name', 'Last_Name', 'Age', 'Phone', 'Email', 'Join_Date', 'Department', 'Region', 'Salary', 'Status', 'Performance_Score', 'Remote_Work'], keep='first')
print("\nJumlah data setelah cleansing:", len(df_cleaned))
df_cleaned #menampilkan data bersih


Jumlah data setelah cleansing: 905


,Employee_ID,First_Name,Last_Name,Age,Phone,Email,Join_Date,Department,Region,Salary,Status,Performance_Score,Remote_Work
0,EMP0001,Bob,Davis,25.0,1651623197,bob.davis@example.com,02 / 04 / 2021,Devops,California,59767.65,Active,Average,True
1,EMP0002,Bob,Brown,30.0,1898471390,bob.brown@example.com,10 / 07 / 2020,Finance,Texas,65304.66,Active,Excellent,True
2,EMP0003,Alice,Jones,30.0,5596363211,alice.jones@example.com,07 / 12 / 2023,Admin,Nevada,88145.90,Pending,Good,True
3,EMP0004,Eva,Davis,25.0,3476490784,eva.davis@example.com,27 / 11 / 2021,Admin,Nevada,69450.99,Inactive,Good,True
4,EMP0005,Frank,Williams,25.0,1586734256,frank.williams@example.com,05 / 01 / 2022,Cloud Tech,Florida,109324.61,Active,Poor,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,EMP0901,Grace,Jones,30.0,2470739200,grace.jones@example.com,16 / 05 / 2021,Finance,Nevada,51990.98,Pending,Excellent,True
901,EMP0902,David,Johnson,30.0,2508261122,david.johnson@example.com,07 / 11 / 2021,Cloud Tech,Texas,100215.06,Inactive,Good,True
902,EMP0903,Charlie,Williams,40.0,1261632487,charlie.williams@example.com,04 / 10 / 2023,Finance,New York,114587.11,Active,Average,False
903,EMP0904,Alice,Garcia,30.0,8995729892,alice.garcia@example.com,16 / 12 / 2024,Hr,Florida,71318.79,Inactive,Good,True


In [279]:
# Simpan hasil cleansing ke file baru
output_path = "/content/drive/MyDrive/Colab Notebooks/Cleaned_Employee_Dataset.xlsx"
df_cleaned.to_excel(output_path, index=False)

print("=== Data Setelah Cleansing ===")
print(df_cleaned.head())
print(f"\nDataset sudah dibersihkan dan disimpan ke: {output_path}")

=== Data Setelah Cleansing ===
  Employee_ID First_Name Last_Name   Age       Phone  \
0     EMP0001        Bob     Davis  25.0  1651623197   
1     EMP0002        Bob     Brown  30.0  1898471390   
2     EMP0003      Alice     Jones  30.0  5596363211   
3     EMP0004        Eva     Davis  25.0  3476490784   
4     EMP0005      Frank  Williams  25.0  1586734256   

                        Email       Join_Date  Department      Region  \
0       bob.davis@example.com  02 / 04 / 2021      Devops  California   
1       bob.brown@example.com  10 / 07 / 2020     Finance       Texas   
2     alice.jones@example.com  07 / 12 / 2023       Admin      Nevada   
3       eva.davis@example.com  27 / 11 / 2021       Admin      Nevada   
4  frank.williams@example.com  05 / 01 / 2022  Cloud Tech     Florida   

      Salary    Status Performance_Score  Remote_Work  
0   59767.65    Active           Average         True  
1   65304.66    Active         Excellent         True  
2   88145.90   Pending   